In [1]:
import enum
import cv2
import traitlets
import time
import ipywidgets.widgets as widgets
import numpy as np

from camera import Camera, bgr8_to_jpeg

In [2]:
# Camera settings *** Needs to match calibration ***

captureWidth  = 820
captureHeight = 616
captureFPS    = 10

In [3]:
print("Reading calibration parameters ......")
cv_file = cv2.FileStorage("data/params_py.xml", cv2.FILE_STORAGE_READ)

Left_Stereo_Map_x = cv_file.getNode("Left_Stereo_Map_x").mat()
Left_Stereo_Map_y = cv_file.getNode("Left_Stereo_Map_y").mat()
Right_Stereo_Map_x = cv_file.getNode("Right_Stereo_Map_x").mat()
Right_Stereo_Map_y = cv_file.getNode("Right_Stereo_Map_y").mat()
cv_file.release()

Reading calibration parameters ......


In [4]:
cameraL = Camera.instance(id=0, width=captureWidth, height=captureHeight, fps=captureFPS)
cameraR = Camera.instance(id=1, width=captureWidth, height=captureHeight, fps=captureFPS)

In [5]:
def adjustGamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
    for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)


    new_image[y,x,c] = np.clip(alpha*image[y,x,c] + beta, 0, 255)

In [6]:
def cameraOn(b):   
    cameraL.start()
    cameraR.start()

def cameraOff(b):
    cameraL.stop()
    cameraR.stop()

def snapPhoto(b):
    global currentL, currentR, currentAnaglyph
    
    currentL = cameraL.value
    currentR = cameraR.value
    dummy=makeAnaglyph(None) 
    
def saveAnaglyph(b):
    global currentL, currentR, currentAnaglyph
    
    timestr = time.strftime("%Y%m%d-%H%M%S")
    file=f"anaglyphs/anaglyph-{timestr}.png"
    cv2.imwrite(file, currentAnaglyph)
    label.value="Anaglyph saved"

In [11]:
def makeAnaglyph(b):
    global currentL, currentR, currentAnaglyph
    
    left = cv2.cvtColor(currentL, cv2.COLOR_BGR2GRAY)
    right = cv2.cvtColor(currentR, cv2.COLOR_BGR2GRAY)

    left = adjustGamma(left, gamma=gamma.value)
    right = adjustGamma(right, gamma=gamma.value)
    
    left = cv2.remap(left,Left_Stereo_Map_x,Left_Stereo_Map_y, cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
    right = cv2.remap(right,Right_Stereo_Map_x,Right_Stereo_Map_y, cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)

    M = np.float32([[1, 0, shift.value],[0, 1, 0]])
    right = cv2.warpAffine(right, M, (right.shape[1], right.shape[0]))
    
    currentAnaglyph = currentL.copy()
    currentAnaglyph[:,:,0] = right[:,:]
    currentAnaglyph[:,:,1] = right[:,:]    
    currentAnaglyph[:,:,2] = left[:,:]

    cropX = 50 + 50   # Second 50 is maximum shift value
    cropY = 50
    currentAnaglyph = currentAnaglyph[0+cropY:-cropY, 0+cropX:-cropX]
    anaglyph.value=bgr8_to_jpeg(currentAnaglyph)

In [13]:
layout=widgets.Layout(width="97%", margin="3px")

anaglyph = widgets.Image(width=820, height=616)
snap   = widgets.Button(description="Take photo", layout=layout, button_style='primary')
save   = widgets.Button(description="Save anaglyph", layout=layout, button_style='primary')
camOn  = widgets.Button(description="Camera on", layout=layout, button_style='success')
camOff = widgets.Button(description="Camera off", layout=layout, button_style='danger')
shift  = widgets.IntSlider(value=25, min=-50, max=50, step=1, description='shift:', orientation='vertical')
gamma  = widgets.FloatSlider(value=1.25, min=0.0, max=2.0, step=0.05, description='gamma:', orientation='vertical')
camera = cameraL.widget(width=309, height=231)
label  = widgets.Label(value="Ready", layout=layout)

sliders = widgets.HBox([gamma, shift])
buttons = widgets.VBox([camOn, camOff,snap, save])
controls = widgets.VBox([camera, sliders, buttons, label])
display(widgets.HBox([controls, anaglyph]))

gamma.observe(makeAnaglyph, names='value')
shift.observe(makeAnaglyph, names='value')
snap.on_click(snapPhoto)
save.on_click(saveAnaglyph)
camOn.on_click(cameraOn)
camOff.on_click(cameraOff)

time.sleep(0.5)
snapPhoto(None)

In [14]:
print(cameraL.value.shape)

(616, 820, 3)
